In [1]:
import platform
import os
if platform.system() == 'Darwin':
    DATA_PATH = "/Users/maltegenschow/Documents/Uni/Thesis/Data.nosync"
    ROOT_PATH = "/Users/maltegenschow/Documents/Uni/Thesis/Thesis"
elif platform.system() == 'Linux':
    DATA_PATH = "/pfs/work7/workspace/scratch/tu_zxmav84-thesis/Data.nosync"
    ROOT_PATH = "/pfs/work7/workspace/scratch/tu_zxmav84-thesis/Thesis"

current_wd = os.getcwd()

In [2]:
import numpy as np
from interfacegan_utils import calculate_boundary
from glob import glob
import pandas as pd
from tqdm import tqdm

In [3]:
latent_input_base_path = f"{DATA_PATH}/Models/InterfaceGAN/Inputs/e4e_00005"
all_paths = glob(f"{latent_input_base_path}/*/*")
all_paths = [path for path in all_paths if not "summary_stats" in path]
all_paths = [elem for elem in all_paths if "price" in elem]

In [4]:
for path in tqdm(all_paths):
    out_dir = path.replace('Inputs', 'Outputs')
    if os.path.exists(out_dir):
        continue

    else:
        accuracies = []
        os.makedirs(out_dir, exist_ok=True)
        for dim in range(16):
            latent_codes_path = f"{path}/latents_dim_{dim}.npy"
            scores_path = f"{path}/target.npy"
            out_name = f'boundary_dim{dim}.npy'
            acc = calculate_boundary(out_dir, out_name, latent_codes_path, scores_path)
            accuracies.append(acc)

        # save result
        attribute = path.split('/')[-2]
        values = path.split('/')[-1].split('_')

        # Crate dataframe with columns 'attribute', 'value1, 'value2', 'accuracy1', 'accuracy2', ...
        result = pd.DataFrame(columns=['attribute'] + ['neg_class', 'pos_class'] + [f'acc_dim_{i}' for i in range(len(accuracies))])
        result.loc[0] = [attribute] + values + accuracies
        result.to_csv(f"{out_dir}/summary_stats.csv", index=False)

100%|██████████| 7/7 [45:34<00:00, 390.65s/it]


In [7]:
summary_paths = glob(f"{latent_input_base_path.replace('Inputs', 'Outputs')}/price/*/summary_stats.csv")

training_results = pd.concat([pd.read_csv(path) for path in summary_paths])
training_results.sort_values('pos_class')

,attribute,neg_class,pos_class,acc_dim_0,acc_dim_1,acc_dim_2,acc_dim_3,acc_dim_4,acc_dim_5,acc_dim_6,acc_dim_7,acc_dim_8,acc_dim_9,acc_dim_10,acc_dim_11,acc_dim_12,acc_dim_13,acc_dim_14,acc_dim_15
0,price,threshold,100,0.688241,0.685870,0.687293,0.688952,0.696064,0.701517,0.688952,0.685870,0.693457,0.698435,0.693694,0.695827,0.691560,0.690375,0.707681,0.690138
0,price,threshold,200,0.831437,0.835467,0.833333,0.828829,0.837364,0.833333,0.830251,0.835941,0.828355,0.844002,0.832859,0.835467,0.834045,0.828592,0.839023,0.827169
0,price,threshold,300,0.906354,0.902798,0.906591,0.907539,0.903983,0.907539,0.914414,0.912755,0.905168,0.911095,0.907065,0.913940,0.910384,0.908013,0.912281,0.901138
0,price,threshold,400,0.948791,0.945235,0.947368,0.946894,0.941204,0.943338,0.945946,0.948080,0.946657,0.951873,0.950450,0.947843,0.947843,0.947131,0.945472,0.949028
0,price,threshold,500,0.964675,0.961830,0.964438,0.964438,0.963016,0.958511,0.964675,0.963016,0.963964,0.962779,0.962541,0.966572,0.962541,0.962067,0.963490,0.967994
0,price,threshold,600,0.979611,0.974158,0.972973,0.976766,0.974158,0.973684,0.974158,0.970365,0.971313,0.973684,0.976766,0.976529,0.974870,0.975818,0.976766,0.971788
0,price,threshold,700,0.981034,0.979611,0.981745,0.983642,0.982693,0.980797,0.979611,0.980085,0.979611,0.973447,0.981982,0.984590,0.983642,0.979137,0.983404,0.983642
